In [1]:
# Libraries
import itertools
from importlib import resources
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import random
import time

from drdt.helper_functions import DecisionRuleCreatorFromDecisionTable, Reduction, R_SR, R_AD, SAlphaStep, SPlus, SMax, NCover, NGreedy
from drdt.algorithms import DynamicProgrammingAlgorithms, A_C_N, A_C_G


# Loading Data

In [2]:
with resources.path('datasets.DecisionRuleSystems', 'DRS_breast_cancer') as dataset_path:
    S = pd.read_csv(dataset_path).applymap(lambda x: str(x) if pd.notnull(x) else x)
S = S.dropna(axis=1, how='all') # Drop the columns with all None
S

/var/folders/06/0m2hb6f927s6rdyhxjrbss740r2h_1/T/ipykernel_27667/2576554060.py:1: DeprecationWarning: path is deprecated. Use files() instead. Refer to https://importlib-resources.readthedocs.io/en/latest/using.html#migrating-from-legacy for migration advice.
  with resources.path('datasets.DecisionRuleSystems', 'DRS_balance_scale') as dataset_path:
/var/folders/06/0m2hb6f927s6rdyhxjrbss740r2h_1/T/ipykernel_27667/2576554060.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  S = pd.read_csv(dataset_path).applymap(lambda x: str(x) if pd.notnull(x) else x)


,right-distance,right-weight,left-distance,left-weight,class
0,1.0,1.0,1.0,1.0,B
1,2.0,NaN,1.0,1.0,R
2,3.0,NaN,1.0,1.0,R
3,4.0,NaN,1.0,1.0,R
4,5.0,NaN,1.0,1.0,R
...,...,...,...,...,...
620,1.0,NaN,5.0,5.0,L
621,2.0,NaN,5.0,5.0,L
622,3.0,NaN,5.0,5.0,L
623,4.0,NaN,5.0,5.0,L


# Dataset Analyses

In [3]:
n = len(S.columns)-1
print(f"Number of features  = {n}")

Number of features  = 4


In [4]:
print(f"Does # of features = d  = {not S.dropna().empty}")

Does # of features = d  = True


In [5]:
# Count non-NaN values for each row
non_nan_counts = S.count(axis=1)

# Find the index
max_non_nan_row_index = non_nan_counts.idxmax()

# Retrieve the row
max_non_nan_row = S.loc[max_non_nan_row_index]

# Number of non-NaN values in the row
max_non_nan_count = non_nan_counts[max_non_nan_row_index]

d = max_non_nan_count - 1

print(f"d = {d}") # remove 1 because of last class column

d = 4


In [6]:
P = S
P_plus = SPlus(P)
B = NCover(P_plus)
print(f"Length of Node Cover for AR = {len(B)}")

Length of Node Cover for AR = 4


In [7]:
P = R_SR(S)
P_plus = SPlus(P)
B = NCover(P_plus)
print(f"Length of Node Cover for SR = {len(B)}")

Length of Node Cover for SR = 4


In [8]:
P = R_AD(S)
P_plus = SPlus(P)
B = NCover(P_plus)
print(f"Length of Node Cover for AD = {len(B)}")

Length of Node Cover for AD = 4


In [9]:
P = S
P_plus = SPlus(P)
B = NGreedy(P_plus)
print(f"Length of Node Cover greedy for AR = {len(B)}")

Length of Node Cover greedy for AR = 1


In [10]:
P = R_SR(S)
P_plus = SPlus(P)
B = NGreedy(P_plus)
print(f"Length of Node Cover greedy for SR = {len(B)}")

Length of Node Cover greedy for SR = 1


In [11]:
P = R_AD(S)
P_plus = SPlus(P)
B = NGreedy(P_plus)
print(f"Length of Node Cover greedy for AD = {len(B)}")

Length of Node Cover greedy for AD = 1


# Combinations

In [12]:
column_values = []

for column in S.columns[:-1]: #ignoring class column
    column_values.append(list(S[column].dropna().unique()))
    
# All possible combinations    
combinations = list(itertools.product(*column_values))

print('Number of Possible combinations =', len(combinations))

Number of Possible combinations = 625


In [13]:
ext_column_values = []

for column in S.columns[:-1]: #ignoring class column
    ext_column_values.append(list(S[column].dropna().unique()) + ['*'])
    
# All possible combinations with extension    
ext_combinations = list(itertools.product(*ext_column_values))

print('Number of Possible ext_combinations =', len(ext_combinations))

# Greedy

### AR

In [14]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="AR")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)

end_time = time.time()
    
print("Greedy AR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|████████████████████████████████████████| 625/625 [00:01<00:00, 476.37it/s]

Greedy AR
(Max Depth, Min Depth, Average Depth, Running time) = (4, 4, 4.0, 1.3235609531402588)


### EAR

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(ext_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="EAR")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)

end_time = time.time()
    
print("Greedy EAR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

### SR

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="SR")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)

end_time = time.time()
    
print("Greedy SR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

### ESR

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(ext_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="ESR")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)

end_time = time.time()
    
print("Greedy ESR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

### AD

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="AD")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)

end_time = time.time()
    
print("Greedy AD")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

### EAD

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(ext_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="EAD")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)

end_time = time.time()
    
print("Greedy EAD")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))